In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**檢查data**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from itertools import accumulate
# 统计句子长度及长度出现的频数
df = pd.read_csv('/content/drive/MyDrive/chinese_sent_data/news_data_1019.csv',index_col=0)
print(df.groupby('label')['label'].count())

df['length'] = df['text'].apply(lambda x: len(x))
len_df = df.groupby('length').count()
sent_length = len_df.index.tolist()
sent_freq = len_df['text'].tolist()
del df

# 绘制句子长度及出现频数统计图
plt.bar(sent_length, sent_freq)
plt.title("Sentence length and frequency statistics chart")
plt.xlabel("Sentence Length")
plt.ylabel("Frequency of sentence length")
plt.savefig("./Sentence length and frequency statistics graph.png")
plt.close()
# 绘制句子长度累积分布函数(CDF)
sent_pentage_list = [(count/sum(sent_freq)) for count in accumulate(sent_freq)]

# 绘制CDF
plt.plot(sent_length, sent_pentage_list)

# 寻找分位点为quantile的句子长度
quantile = 0.91
#print(list(sent_pentage_list))
for length, per in zip(sent_length, sent_pentage_list):
    if round(per, 2) == quantile:
        index = length
        break
print("\n分位点为%s的句子长度:%d." % (quantile, index))

# 绘制句子长度累积分布函数图
plt.plot(sent_length, sent_pentage_list)
plt.hlines(quantile, 0, index, colors="c", linestyles="dashed")
plt.vlines(index, 0, quantile, colors="c", linestyles="dashed")
plt.text(0, quantile, str(quantile))
plt.text(index, 0, str(index))
plt.title("Sentence length cumulative distribution function graph")
plt.xlabel("Sentence Length")
plt.ylabel("Sentence Length Cumulative Frequency")
plt.savefig("./Sentence length cumulative distribution function graph.png")
plt.close()

label
-2    1660
-1    1956
 0    1872
 1    2239
 2    3105
Name: label, dtype: int64

分位点为0.91的句子长度:402.


In [ ]:
df.head(5)

,text,label,length
Unnamed: 0,,,
4246,"中國證券網訊（記者 駱民）京天利(行情300399,診股)公告，持公司股份1,436.40萬...",0,132
16765,中國交建子公司工地發生坍塌事故 12死2失聯,-2,22
2331,"12月12日訊 迪威迅(300167)週三晚公告,公司今日收到控股股東北京安策的函告,因近期...",-2,201
2724,"遠大控股(行情000626,診股)6月28日晚公告，持股14.56%的股東金控資本擬未來6個...",1,75
602,"金融界網站訊 哈森股份(行情603958,診股)1月22日晚間公告稱，經財務部門初步測算，預...",-2,482


**建立模型字典**

In [ ]:
# -*- coding: utf-8 -*-

import pickle
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 匯入資料
# 檔案的資料中，特徵為text, 類別為label.
def load_data(filepath, input_shape=20):
    df = pd.read_csv(filepath)

    # 標籤及詞彙表
    labels, vocabulary = list(df['label'].unique()), list(df['text'].unique())

    # 構造字元級別的特徵
    string = ''
    for word in vocabulary:
        string += word

    vocabulary = set(string)

    # 字典列表
    word_dictionary = {word: i+1 for i, word in enumerate(vocabulary)}
    with open('word_dict.pk', 'wb') as f:
        pickle.dump(word_dictionary, f)
    inverse_word_dictionary = {i+1: word for i, word in enumerate(vocabulary)}
    label_dictionary = {label: i for i, label in enumerate(labels)}
    with open('label_dict.pk', 'wb') as f:
        pickle.dump(label_dictionary, f)
    output_dictionary = {i: labels for i, labels in enumerate(labels)}

    vocab_size = len(word_dictionary.keys()) # 詞彙表大小
    label_size = len(label_dictionary.keys()) # 標籤類別數量

    # 序列填充，按input_shape填充，長度不足的按0補充
    x = [[word_dictionary[word] for word in sent] for sent in df['text']]
    x = pad_sequences(maxlen=input_shape, sequences=x, padding='post', value=0)
    y = [[label_dictionary[sent]] for sent in df['label']]
    y = [np_utils.to_categorical(label, num_classes=label_size) for label in y]
    y = np.array([list(_[0]) for _ in y])

    return x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary

# 建立深度學習模型， Embedding + LSTM + Softmax.
def create_LSTM(n_units, input_shape, output_dim, filepath):
    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath)
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size + 1, output_dim=output_dim,
                        input_length=input_shape, mask_zero=True))
    model.add(LSTM(n_units, input_shape=(x.shape[0], x.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(label_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    plot_model(model, to_file='./model_lstm.png', show_shapes=True)
    model.summary()

    return model

# 模型訓練
def model_train(input_shape, filepath, model_save_path):

    # 將資料集分為訓練集和測試集，佔比為9:1
    # input_shape = 100
    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath, input_shape)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.1, random_state = 42)

    # 模型輸入引數，需要自己根據需要調整
    n_units = 100
    batch_size = 32
    epochs = 18
    output_dim = 20

    # 模型訓練
    lstm_model = create_LSTM(n_units, input_shape, output_dim, filepath)
    lstm_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1)

    # 模型儲存
    lstm_model.save(model_save_path)

    N = test_x.shape[0]  # 測試的條數
    predict = []
    label = []
    for start, end in zip(range(0, N, 1), range(1, N+1, 1)):
        sentence = [inverse_word_dictionary[i] for i in test_x[start] if i != 0]
        y_predict = lstm_model.predict(test_x[start:end])
        label_predict = output_dictionary[np.argmax(y_predict[0])]
        label_true = output_dictionary[np.argmax(test_y[start:end])]
        # print(''.join(sentence), label_true, label_predict) # 輸出預測結果
        predict.append(label_predict)
        label.append(label_true)

    acc = accuracy_score(predict, label) # 預測準確率
    print('模型在測試集上的準確率為: %s.' % acc)

# if __name__ == '__main__':
#     filepath = '/content/train.csv'
#     input_shape = 291
#     model_save_path = './corpus_model.h5'
#     model_train(input_shape, filepath, model_save_path)

In [ ]:
filepath = '/content/drive/MyDrive/chinese_sent_data/news_data_1019.csv'

df = pd.read_csv(filepath)
vocabulary = list(df['text'].unique())
# 構造字元級別的特徵
string = ''
for word in vocabulary:
    string += word

vocabulary = set(string)

# 字典列表
word_dictionary = {word: i+1 for i, word in enumerate(vocabulary)}
with open('word_dict.pk', 'wb') as f:
    pickle.dump(word_dictionary, f)

In [ ]:
len(word_dictionary.keys())

3458

**使用模型預測情感分數**

In [ ]:
# 合併16-21年新聞
import pandas as pd
import glob

path = '/content/drive/MyDrive/News_Anue' # use your path
all_files = glob.iglob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, dtype=str)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame

In [ ]:
# 去除沒有在字典裏面的字
for sent in frame['summary']:
    for word in sent:
        if word in word_dictionary:
            pass
        else:
            frame['summary'] = frame['summary'].str.replace(word, "")

In [ ]:
frame

In [ ]:
# -*- coding: utf-8 -*-

# Import the necessary modules
import pickle
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences


label_list = []
# 匯入字典
with open('word_dict.pk', 'rb') as f:
    word_dictionary = pickle.load(f)
with open('/content/drive/MyDrive/chinese_sent_dict/label_dict.pk', 'rb') as f:
    output_dictionary = pickle.load(f)

for sent in frame["summary"]:
    try:
        # 資料預處理
        input_shape = 300
        sent = sent
        x = [[word_dictionary[word] for word in sent]]
        x = pad_sequences(maxlen=input_shape, sequences=x, padding='post', value=0)

        # 載入模型
        model_save_path = '/content/drive/MyDrive/Sentiment_Model_NLP/corpus_model_4.h5'
        lstm_model = load_model(model_save_path)

        # 模型預測
        y_predict = lstm_model.predict(x)
        label_dict = {v:k for k,v in output_dictionary.items()}
        print('輸入語句: %s' % sent)
        print('情感預測結果: %s' % label_dict[np.argmax(y_predict)])
        frame['result']=(label_dict[np.argmax(y_predict)])
    except KeyError as err:
        print("您輸入的句子有漢字不在詞彙表中，請重新輸入！")
        print("不在詞彙表中的單詞為：%s." % err)
        frame['result']=""

串流輸出內容已截斷至最後 5000 行。
輸入語句: 台泥 (1101-TW) 今 (2) 日宣布，啟動 2021 年徵才計畫，包括儲備幹部 (MA) 及各領域職缺，台泥董事長張安平指出，疫情改變了市場的格局，但無法改變這幾年來台泥所訂下的目標，布局循環經濟、邁向國際化的腳步不停歇。
情感預測結果: 2
輸入語句: 世界衛生組織 (WHO) 公布今年北半球流感疫苗的病毒候選株，國光生 (4142-TW) 表示，全球疫苗廠將依據此結果生產流感疫苗，目前已備妥 SPF(無特定病原) 雞蛋進行繼代培養，並挑選高產率病毒株，同時生產流感疫苗原液。
情感預測結果: 2
輸入語句: 義集團旗下不鋼廠燁聯 (9957-TW) 宣布，與朝陽科技大學簽訂合作意向書，透過產學合作，聚焦高端、環保等材料，強化藍海市場佈局。
情感預測結果: 2
輸入語句: 機上盒晶片廠揚智 (3041-TW) 今 (2) 日公布去年財報，去年第四季稅後虧損 0.7 億元，每股稅後虧損 0.37 元，全年稅後虧損 2.39 億元，較前年收斂，每股稅後虧損 1.26 元；揚智去年受惠財務結構改善，以及機上盒市占率增加
情感預測結果: 2
輸入語句: 光收發模組廠眾達 - KY (4977-TW) 今 (2) 日公布去年財報，受惠 5G、資料中心需求維持高檔，全年稅後純益達 5.38 億元、年增 116.9%，每股純益達 8.02 元，創下歷史新高，眾達今也通過去年盈餘分配。
情感預測結果: 2
輸入語句: 半導體通路商弘憶股 (3312-TW) 今 (2) 日公布 1 月自結，稅後純益 0.16 億元，年增 1003%，每股稅後純益 0.13 元；隨著既有產品線拓展有成，加上新代理產品線發酵，弘憶股 1 月營收創近 5 年半新高，獲利也隨著營收規模放大
情感預測結果: 2
輸入語句: 台股攻上「萬六」加持下，前 2 月權證發行檔數與交易金額均較去年同期增加，累計交易金額更逼近千億元。
情感預測結果: -2
輸入語句: 萬寶投顧陳子榕指出，歷經 228 連假，美國 10 年期公債殖利率終於跌破 1.5％關卡，讓市場緊張氣氛稍微緩和。就美國聯準會的說法，目前市場通膨是景氣復甦的正常表現，而且至少需要三年以上才有辦法將通膨拉到美國聯準會的目標 2％。
情感預測結果: 2
輸入語句: 小資族也可定期定額買美股 ETF

In [ ]:
frame.groupby(['result']).size().reset_index(name='counts')

In [ ]:
frame.to_csv('anue_news_labeled.csv', index=False)

#### **使用模型預測情感分數_改**

In [ ]:
import pandas as pd
temp = pd.read_csv("/content/drive/MyDrive/BDSE21_Team1/Final_Project/Dataset_A/News_Anue_Arrange_NLP/News_Anue_Arrange_NLP.csv",index_col=0)
# temp= temp.sample(100)

In [ ]:
# 去除沒有在字典裏面的字
import pickle
with open('/content/drive/MyDrive/Sentiment_analysis/LTSM/Shen/word_dict.pk', 'rb') as f:
    word_dictionary = pickle.load(f)
for sent in temp["news"]:
    for word in sent:
        if word in word_dictionary:
            pass
        else:
            temp["news"] = temp["news"].str.replace(word, "")

In [ ]:
# Import the necessary modules
import pickle
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# 匯入字典
# with open('/content/drive/MyDrive/Sentiment_analysis/LTSM/Shen/word_dict.pk', 'rb') as f:
#     word_dictionary = pickle.load(f)
with open('/content/drive/MyDrive/chinese_sent_dict/label_dict.pk', 'rb') as f:
    output_dictionary = pickle.load(f)
label_list = []
# 載入模型
model_save_path = '/content/drive/MyDrive/Sentiment_analysis/LTSM/Shen/corpus_model_4.h5'
lstm_model = load_model(model_save_path)
for sent in temp["news"]:
  try:
      # 資料預處理
      input_shape = 260
      sent = sent
      x = [[word_dictionary[word] for word in sent]]
      x = pad_sequences(maxlen=input_shape, sequences=x, padding='post', value=0)
      # 模型預測
      y_predict = lstm_model.predict(x)
      label_dict = {v:k for k,v in output_dictionary.items()}
      print('輸入語句: %s' % sent)
      print('情感預測結果: %s' % label_dict[np.argmax(y_predict)])
      label_list.append(label_dict[np.argmax(y_predict)])
  except KeyError as err:
      print("您輸入的句子有漢字不在詞彙表中，請重新輸入！")
      print("不在詞彙表中的單詞為：%s." % err)
      label_list.append('')

串流輸出內容已截斷至最後 5000 行。
輸入語句: 台股今 (26) 日在航運類股領軍之下，終場收在 16643.69 點，成功站回月線，三大法人也同站買方，合計買超 47.75 億元，外資也終止連二賣，轉為買超，大舉搶進航運三雄，其中，買超陽明 4.3 萬張。
情感預測結果: -2
輸入語句: 中國信託再添一內勤員工確診 中信金已 4 人染疫《財訊》報導指出，去年此時全球慘痛封境，台灣歌舞昇平；如今大來了、疫情蔓延了、市場重挫了，才揭開台灣其實從未誠實面對環境 (E)、社會 (S)、治理 (G) 風險的真相。台股今 (26) 日在航運類股領軍之下，終場收在 16643.69 點，成功站回月線，三大法人也同站買方，合計買超 47.75 億元，外資也終止連二賣，轉為買超，大舉搶進航運三雄，其中，買超陽明 4.3 萬張。
情感預測結果: -2
輸入語句: 連接器端子廠健和興 (3003-TW) 今 (26) 日召開法說，在原料銅價高檔下，公司表示，將啟動第二波漲價，預計漲幅 1-2 成，而在反應成本下，第二季除了營收將季增外，毛利率也可望維持水準。
情感預測結果: -2
輸入語句: 台股今 (26) 日在航運類股領軍之下，終場收在 16643.69 點，成功站回月線，三大法人也同站買方，合計買超 47.75 億元，外資也終止連二賣，轉為買超，大舉搶進航運三雄，其中，買超陽明 4.3 萬張。
情感預測結果: -2
輸入語句: 蘋果開發者大會 (WWDC) 將在台灣時間 6 月 8 日凌晨舉行，讓台廠供應鏈添增想像，其中，混合實境 (MR) 頭戴式裝置近來獲外資高度關注，並點名大立光、玉晶光等光學族群擁技術優勢，玉晶光今 (26) 日股價逆勢翻
情感預測結果: -2
輸入語句: 受惠遠距商機、宅經濟需求帶動，全球個人電腦裝置動能強勁，市調機構 IDC 最新預期，雖然受半導體等零組件缺料影響，但今年全年出貨仍將上看 3.57 億台、年增 18.1%，明年則將回落，長期則將保持穩定成長。
情感預測結果: -2
輸入語句: 記憶體模組廠威剛 (3260-TW) 今 (26) 日公告，23 日偵測到部分資通系統遭病毒攻擊，立即與資安專業人員合作清除病毒，此事件對公司營運尚無造成重大影響。
情感預測結果: -2
輸入語句: 蘋果開發者大會 (WWDC) 將在台灣時間 6 月 8 日

In [ ]:
temp['lstm_label'] = label_list;temp

,date,code,news,nlp,lstm_label
0,2016-04-11,1301,台塑四寶各公司每股盈餘部分，台化(1326-TW)每股盈餘1.16元最高，台塑化(6505-...,1,-1
1,2016-04-11,1303,台塑四寶各公司每股盈餘部分，台化(1326-TW)每股盈餘1.16元最高，台塑化(6505-...,1,-1
2,2016-04-11,1326,台塑四寶各公司每股盈餘部分，台化(1326-TW)每股盈餘1.16元最高，台塑化(6505-...,1,-1
3,2016-04-11,1338,中國汽車內飾零組件廠F-廣華(1338-TW)3月在中國新車市場銷售趨緩下，單月營收為6.9...,0,1
4,2016-04-11,1599,宏佳騰Q1業績創同期新高 將攜手中信兄弟行銷,1,2
...,...,...,...,...,...
100750,2021-06-30,6488,先前提到隨時可能過高的台股，果然在 6/29 日來到 17713 點，改寫歷史新高；目前在半...,1,-2
100751,2021-06-30,6505,台股今 (30) 日續創歷史新高，攻抵 17795 點，航海王攻勢再起，其中，貨櫃航商長榮 ...,1,-2
100752,2021-06-30,6669,英特爾 (INTC-US) 受 10 奈米製程延誤，預期代號為 Sapphire Rapid...,0,-2
100753,2021-06-30,8415,美國鋁捲板通路商大成鋼 (2027-TW) 今 (30) 日公告 5 月稅後純益 6.82 ...,1,-2


In [ ]:
temp.to_csv("/content/drive/MyDrive/chinese_sent_dict/anue_news_labeled.csv", index=False)